# How to deploy the SEA-LION v4 instruct (Gemma-SEA-LION-v4-27B-IT) for inference using Amazon SageMakerAI with LMI v15 powered by vLLM 0.8.4
**Recommended kernel(s):** This notebook can be run with any Amazon SageMaker Studio kernel.

In this notebook, you will learn how to deploy the Gemma SEA-LION v4 27 B instruct model (HuggingFace model ID: [aisingapore/Gemma-SEA-LION-v4-27B-IT](https://huggingface.co/aisingapore/Gemma-SEA-LION-v4-27B-IT)) using Amazon SageMaker AI. The inference image will be the SageMaker-managed [LMI (Large Model Inference) v15 powered by vLLM 0.8.4](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-container-docs.html) Docker image. LMI images features a [DJL serving](https://github.com/deepjavalibrary/djl-serving) stack powered by the [Deep Java Library](https://djl.ai/). 

The SEA-LION v4 (Gemma3-based) models are multimodal, handling text and image input and generating text output, with open weights for both pre-trained variants and instruction-tuned variants. The base Gemma 3 model has a large, 128K context window, multilingual support in over 140 languages, and is available in more sizes than previous versions. Gemma 3 models are well-suited for a variety of text generation and image understanding tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as laptops, desktops or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

### License agreement
* This model is gated on HuggingFace, please refer to the original [model card](https://huggingface.co/google/gemma-3-27b-it) for license.
* This notebook is a sample notebook and not intended for production use.

### Execution environment setup
This notebook requires the following third-party Python dependencies:
* AWS [`sagemaker`](https://sagemaker.readthedocs.io/en/stable/index.html) with a version greater than or equal to 2.242.0

Let's install or upgrade these dependencies using the following command:

In [ ]:
%pip install -Uq sagemaker

### Setup

In [2]:
import sagemaker
import boto3
import logging
import time
from sagemaker.session import Session
from sagemaker.s3 import S3Uploader

print(sagemaker.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
2.251.1


In [3]:
try:
    role = sagemaker.get_execution_role()
    sagemaker_session  = sagemaker.Session()
    
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [4]:
HF_MODEL_ID = "aisingapore/Gemma-SEA-LION-v4-27B-IT"

base_name = HF_MODEL_ID.split('/')[-1].replace('.', '-').lower()
model_lineage = HF_MODEL_ID.split("/")[0]
print(f'base_name = {base_name}')

base_name = gemma-sea-lion-v4-27b-it


## Configure Model Serving Properties

Now we'll create a `serving.properties` file that configures how the model will be served. This configuration is crucial for optimal performance and memory utilization.

Key configurations explained:
- **Engine**: Python backend for model serving
- **Model Settings**:
  -  Using gemma-sea-lion-v4-27b-it 
  - Maximum sequence length of 32768 tokens
  - model loading timeout of 1200 seconds (20 minutes)
- **Performance Optimizations**:
  - Tensor parallelism across all available GPUs
  - Max rolling batch size of 16 for efficient batching
  
#### Understanding KV Cache and Context Window

The `max_model_len` parameter controls the maximum sequence length the model can handle, which directly affects the size of the KV (Key-Value) cache in GPU memory.

1. Start with a conservative value (8192 has been tested successfully with g5.12xlarge, 32768 with g6e.48xlarge)
2. Monitor GPU memory usage
3. Incrementally increase if memory permits
4. Target the model's full context window 

In [5]:
# Create the directory that will contain the configuration files
from pathlib import Path

model_dir = Path('config')
model_dir.mkdir(exist_ok=True)

If you are deploying a model hosted on the HuggingFace Hub, you must specify the `option.model_id=<hf_hub_model_id>` configuration. When using a model directly from the hub, we recommend you also specify the model revision (commit hash or branch) via `option.revision=<commit hash/branch>`. 

Since model artifacts are downloaded at runtime from the Hub, using a specific revision ensures you are using a model compatible with package versions in the runtime environment. Open Source model artifacts on the hub are subject to change at any time. These changes may cause issues when instantiating the model (updated model artifacts may require a newer version of a dependency than what is bundled in the container). If a model provides custom model (modeling.py) and/or custom tokenizer (tokenizer.py) files, you need to specify option.trust_remote_code=true to load and use the model.

In [6]:
config = f"""engine=Python
option.async_mode=true
option.rolling_batch=disable
option.entryPoint=djl_python.lmi_vllm.vllm_async_service
option.tensor_parallel_degree=max
option.model_loading_timeout=1200
fail_fast=true
option.max_model_len=8192
option.max_rolling_batch_size=16
option.trust_remote_code=true
option.model_id={HF_MODEL_ID}
option.revision=main
"""

# If you have copied the model data from HuggingFace to an S3 bucket, you can replace
# option.model_id={HF_MODEL_ID}
# with
# option.model_id={model_s3_uri}

with open("config/serving.properties", "w") as f:
    f.write(config)

In [7]:
# Check that the file config/serving.properties was generated properly
!pygmentize config/serving.properties

engine=Python
option.async_mode=true
option.rolling_batch=disable
option.entryPoint=djl_python.lmi_vllm.vllm_async_service
option.tensor_parallel_degree=max
option.model_loading_timeout=1200
fail_fast=true
option.max_model_len=8192
option.max_rolling_batch_size=16
option.trust_remote_code=true
option.model_id=aisingapore/Gemma-SEA-LION-v4-27B-IT
option.revision=main


**Best Practices**:
>
> **Store Models in Your Own S3 Bucket**
For production use-cases, always download and store model files in your own S3 bucket to ensure validated artifacts. This provides verified provenance, improved access control, consistent availability, protection against upstream changes, and compliance with organizational security protocols.
>
>**Separate Configuration from Model Artifacts**
> The LMI container supports separating configuration files from model artifacts. While you can store serving.properties with your model files, placing configurations in a distinct S3 location allows for better management of all your configurations files.
>
> When your model and configuration files are in different S3 locations, set `option.model_id=<s3_model_uri>` in your serving.properties file, where `s3_model_uri` is the S3 object prefix containing your model artifacts.

#### Optional configuration files

(Optional) You can also specify a `requirements.txt` to install additional libraries.

### Upload config files to S3
SageMaker AI allows us to provide [uncompressed](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-uncompressed.html) files. Thus, we directly upload the folder that contains `serving.properties` to s3
> **Note**: The default SageMaker bucket follows the naming pattern: `sagemaker-{region}-{account-id}`

In [8]:
from sagemaker.s3 import S3Uploader

sagemaker_default_bucket = sagemaker_session.default_bucket()

config_files_uri = S3Uploader.upload(
    local_path="config",
    desired_s3_uri=f"s3://{sagemaker_default_bucket}/lmi/{base_name}/config-files"
)

In [9]:
# print(f"code_model_uri: {config_files_uri}")

## Configure Model Container and Instance

For deploying Gemma-SEA-LION-v4-27B-IT, we'll use:
- **LMI (Deep Java Library) Inference Container**: A container optimized for large language model inference
- **[G5 Instances](https://aws.amazon.com/ec2/instance-types/g5/)**: AWS's GPU instance type powered by NVIDIA A10G Tensor Core GPUs, or **[G6e Instances](https://aws.amazon.com/ec2/instance-types/g6e/)**: AWS's GPU instance type powered by NVIDIA L40S Tensor Core GPUs 

Key configurations:
- The container URI points to the DJL inference container in ECR (Elastic Container Registry)
- We use `ml.g5.12xlarge` instance which offers:
  - 4 NVIDIA A10G Tensor Core GPUs
  - 96 GB of total GPU memory (24 GB of memory per GPU)
  - up to 40 Gbps of network bandwidth (16 Gbps of EBS bandwidth)
  - up to 192 GB of system memory
  - and up to 3.8 TB of local NVMe SSD storage.
- We can also use `ml.g6e.48xlarge` instance which offers:
  - 8 NVIDIA L40S Tensor Core GPUs
  - 384 GB of total GPU memory (48 GB of memory per GPU)
  - up to 400 Gbps of network bandwidth (60 Gbps of EBS bandwidth)
  - up to 1.536 TB of system memory
  - and up to 7.6 TB of local NVMe SSD storage.

> **Note**: The region in the container URI should match your AWS region.

Note the following costs of running ml.g5.12xlarge, and ml.g6e.48xlarge instances in us-east-1 (SageMaker Realtime Inference):

Instance Type     | Cost/hr   | Cost/day  | Cost/month
------------------|-----------|-----------|-------------
`ml.g5.12xlarge`  | USD  7.09 |USD 170.16 |USD  5,175.70
`ml.g6e.48xlarge` | USD 37.664|USD 903.936|USD 27,494.72

If you are not using the instances 24/7, it is highly recommended that you consider deleting the endpoints, or choosing an option such as [SageMaker Scale down to zero feature](https://docs.aws.amazon.com/sagemaker/latest/dg/endpoint-auto-scaling-zero-instances.html) as mentioned in this blog post: [Unlock cost savings with the new scale down to zero feature in SageMaker Inference](https://aws.amazon.com/blogs/machine-learning/unlock-cost-savings-with-the-new-scale-down-to-zero-feature-in-amazon-sagemaker-inference/)

In [ ]:
# gpu_instance_type = "ml.g6e.48xlarge"
gpu_instance_type = "ml.g5.12xlarge"

In [11]:
image_uri = "763104351884.dkr.ecr.{}.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128".format(sagemaker_session.boto_session.region_name)
print(image_uri)

763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128


## Create SageMaker Model

Now we'll create a SageMaker Model object that combines our:
- Container image (LMI)
- Model artifacts (configuration files)
- IAM role (for permissions)

This step defines the model configuration but doesn't deploy it yet. The Model object represents the combination of:

1. **Container Image** (`image_uri`): DJL Inference optimized for LLMs
2. **Model Data** (`model_data`): Our configuration files in S3
3. **IAM Role** (`role`): Permissions for model execution

### Required Permissions
The IAM role needs:
- S3 read access for model artifacts
- CloudWatch permissions for logging
- ECR permissions to pull the container

#### HUGGING_FACE_HUB_TOKEN
This is not required for Gemma-SEA-LION-v4-27B-IT. Note that it is required for Gemma-3-27B-Instruct, which is a gated model. If you deploy model files hosted on the Hub, you need to provide your HuggingFace token as environment variable. This enables SageMaker AI to download the files at runtime.

In [12]:
# Specify the S3 URI for your uncompressed config files
model_data = {
    "S3DataSource": {
        "S3Uri": f"{config_files_uri}/",
        "S3DataType": "S3Prefix",
        "CompressionType": "None"
    }
}

In [13]:
HUGGING_FACE_HUB_TOKEN = "hf_ (fill in you Hugging Face Hub token here)"

In [14]:
from sagemaker.utils import name_from_base
from sagemaker.model import Model

model_name = name_from_base(base_name, short=True)
model_name

'gemma-sea-lion-v4-27b-it-250901-0658'

In [15]:
# Create model
# sealion_4_model = Model(
#     name = model_name,
#     image_uri=image_uri,
#     model_data=model_data,  # Path to uncompressed code files
#     role=role,
#     env={
#         "HF_TASK": "Image-Text-to-Text",
#         "OPTION_LIMIT_MM_PER_PROMPT": "image=2", # Limit the number of images that can be sent per prompt
#         "HUGGING_FACE_HUB_TOKEN": HUGGING_FACE_HUB_TOKEN # HF Token for gated models
#     },
# )
sealion_4_model = Model(
    name = model_name,
    image_uri=image_uri,
    model_data=model_data,  # Path to uncompressed code files
    role=role,
    env={
        "HF_TASK": "Image-Text-to-Text",
        "OPTION_LIMIT_MM_PER_PROMPT": "image=2" # Limit the number of images that can be sent per prompt
    }
)

## Deploy Model to SageMaker Endpoint

Now we'll deploy our model to a SageMaker endpoint for real-time inference. This is a significant step that:
1. Provisions the specified compute resources (G6e instance)
2. Deploys the model container
3. Sets up the endpoint for API access

### Deployment Configuration
- **Instance Count**: 1 instance for single-node deployment
- **Instance Type**: `ml.g5.12xlarge` for high-performance inference

> ⚠️ **Important**: 
> - Deployment can take up to 15 minutes
> - Monitor the CloudWatch logs for progress

The following may take approximately 11-12 minutes (or more)

In [16]:
%%time

from sagemaker.utils import name_from_base

endpoint_name = name_from_base(base_name, short=True)

try:
    sealion_4_model.deploy(
        endpoint_name=endpoint_name,
        initial_instance_count=1,
        instance_type=gpu_instance_type
    )
except Exception as e:
    print(f"Exception: {e}")

----------------------!CPU times: user 440 ms, sys: 35.9 ms, total: 476 ms
Wall time: 11min 33s


### Use the code below to create a predictor from an existing endpoint and make inference

In [17]:
from sagemaker.serializers import JSONSerializer, IdentitySerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor

In [18]:
# Option 1: Automatically retrieve the first endpoint (if this is your only endpoint)
sagemaker_client = boto3.client('sagemaker')
response = sagemaker_client.list_endpoints()
endpoint_names = [ endpoint['EndpointName'] for endpoint in response['Endpoints'] ]
if len(endpoint_names):
    endpoint_name = endpoint_names[0]
    print(f'Using endpoint: {endpoint_name}')

# Option 2: Set the endpoint name manually (Uncomment below to use Option 2)
# endpoint_name = "gemma-3-27b-it-... (replace with your enpoint name)"

Using endpoint: gemma-sea-lion-v4-27b-it-250901-0659


In [19]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

## Text only Inference

In [20]:
def invoke_sealion(prompt: str, print_response=True, **kwargs):
    payload = {
        "messages" : [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "max_tokens": 500,
        "temperature": 0.1,
        "top_p": 0.9,
    }
    for k in kwargs:
        if k in ['max_tokens', 'temperature', 'top_p']:
            payload[k] = kwargs[k]
    response = predictor.predict(payload)
    
    if print_response:
        # Print usage statistics
        usage = response['usage']
        print(response['choices'][0]['message']['content'].strip())
        print(f"=== Token Usage: {usage['prompt_tokens']} (prompt), {usage['completion_tokens']} (completions), {usage['total_tokens']} (total) ===")
    return response['choices'][0]['message']['content']

In [21]:
_ = invoke_sealion("Write me a poem about Machine Learning.", max_tokens=500, temperature=0.1, top_p=0.9)

## The Algorithm's Dream

No longer coded, line by rigid line,
A different path, a future intertwined.
Machine Learning wakes, a nascent mind,
To learn from data, of every kind.

It sifts through oceans, a digital tide,
Of patterns hidden, where truths reside.
From labeled examples, a knowledge gleaned,
A predictive power, subtly seen.

Regression's curve, a gentle slope,
Classification's choice, fueled by hope.
Clustering's dance, to group and define,
Uncovering structures, beautifully aligned.

Neural networks bloom, a complex array,
Layers connecting, day by day.
Backpropagation's flow, a constant quest,
To minimize error, and be the best.

From spam detection to faces it knows,
From medical diagnoses to where the river flows.
It learns to translate, to write and to see,
A growing intelligence, for you and for me.

But caution whispers, a thoughtful plea,
Bias can linger, for all to see.
Transparency needed, a guiding light,
To build with fairness, and do what is right.

So let the 

In [22]:
SAMPLE_PROMPTS = [
    """Terjemahkan teks berikut ini ke dalam Bahasa Inggris. Teks: Anak laki-laki ini, yang secara teknis tidak diijinkan untuk memiliki akun situs ini untuk tiga tahun mendatang,menemukan sebuah bug (kesalahan akibat ketidaksempurnaan desain) yang memungkinkan dia menghapus komentar yang dibuat oleh pengguna lain. Masalah ini dengan “cepat” diperbaiki setelah ditemukan, demikian keterangan Facebook, perusahaan media sosial yang memiliki Instagram. Jani kemudian dibayar - yang membuat dia sebagai anak yang termuda yang pernah menerima hadiah atas penemuan bug ini. Setelah menemukan kekurangan itu pada Februari, dia mengirim email ke Facebook. Beli sepeda dan peralatan sepak bola Sejumlah ahli teknik keamanan di perusahaan itu telah membuat akun uji coba kepada Jani untuk membuktikan teorinya - dan dia dapat melakukannya. Anak laki-laki ini, dari Helsinki, mengatakan kepada koran Finlandia Iltalehti, dia berencana untuk menggunakan uang itu untuk membeli sepeda baru, peralatan sepak bola dan komputer untuk saudara laki-lakinya. Facebook mengatakan kepada BBC, telah membayar $4.3 juta sebagai hadiah bagi yang menemukan bug sejak 2011. Banyak perusahaan menawarkan sebuah insentif keuangan bagi profesional keamanan - dan anak-anak muda, yang menyampaikan kekurangan itu kepada perusahaan, dibandingkan menjualnya ke pasar gelap. Terjemahan:""",
    """Apa sentimen dari kalimat berikut ini? Kalimat: Buku ini sangat membosankan. Jawaban:""",
    """Anda akan diberikan sebuah teks dan pertanyaan. Jawablah pertanyaan tersebut berdasarkan teks yang tersedia. > Teks: “Isyana lahir di Bandung pada 2 Mei 1993. Dia menghabiskan masa kecilnya di berbagai lokasi, karena orang tuanya bekerja & melanjutkan studi mereka di Belgia. Namun, pada usia 7 tahun keluarganya pindah ke Bandung, Indonesia. Isyana adalah putri bungsu dari pasangan Luana Marpanda, seorang guru musik, dan Sapta Dwikardana, Ph.D seorang dosen dan terapis (grafologis). Ia memiliki kakak perempuan bernama Rara Sekar Larasati, yang juga merupakan vokalis band bernama Banda Neira. Dibesarkan dalam keluarga pendidik, Isyana diperkenalkan ke dunia musik pada usia 4 tahun oleh ibunya. Isyana telah menguasai sejumlah instrumen. Termasuk piano, electone, flute, biola, dan saksofon.” > Pertanyaan: Siapa nama orang tua Isyana?""",
    """Sebutkan persamaan dan perbedaan antara gado-gado, ketoprak dan karedok""",
    """Jelaskan budaya Indonesia menyapa orang yang lebih tua?""",
    """Jelaskan budaya pulang kampung ketika lebaran?""",
    """Sebutkan berbagai jenis kopi dan karakteristik rasanya yang berasal dari Indonesia"""   
]

In [23]:
for prompt in SAMPLE_PROMPTS:
    print(f"##### Prompt: {prompt} #####")
    print(f"##### Response #####")
    _ = invoke_sealion(prompt, max_tokens=1024, temperature=0.1, top_p=0.9)
    print()

##### Prompt: Terjemahkan teks berikut ini ke dalam Bahasa Inggris. Teks: Anak laki-laki ini, yang secara teknis tidak diijinkan untuk memiliki akun situs ini untuk tiga tahun mendatang,menemukan sebuah bug (kesalahan akibat ketidaksempurnaan desain) yang memungkinkan dia menghapus komentar yang dibuat oleh pengguna lain. Masalah ini dengan “cepat” diperbaiki setelah ditemukan, demikian keterangan Facebook, perusahaan media sosial yang memiliki Instagram. Jani kemudian dibayar - yang membuat dia sebagai anak yang termuda yang pernah menerima hadiah atas penemuan bug ini. Setelah menemukan kekurangan itu pada Februari, dia mengirim email ke Facebook. Beli sepeda dan peralatan sepak bola Sejumlah ahli teknik keamanan di perusahaan itu telah membuat akun uji coba kepada Jani untuk membuktikan teorinya - dan dia dapat melakukannya. Anak laki-laki ini, dari Helsinki, mengatakan kepada koran Finlandia Iltalehti, dia berencana untuk menggunakan uang itu untuk membeli sepeda baru, peralatan se

## Multimodality

SEA LION v4 models (based on Gemma 3) are multimodal, handling text and image input and generating text output, with open weights for both pre-trained variants and instruction-tuned variants.

#### single image

In [24]:
from IPython.display import Image as IPyImage
IPyImage(url="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG", height=300, width= 300)

In [25]:
def invoke_sealion_multimodal(prompt: str, image_url: str, system_prompt="You are a helpful assistant.", print_response=True, **kwargs):
    payload = {
        "messages": [
            {
              "role": "system",
              "content": [
                  {
                      "type": "text",
                      "text": system_prompt
                  }
              ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url", 
                        "image_url": {
                            "url": image_url
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        "max_tokens": 500,
        "temperature": 0.1,
        "top_p": 0.9,
    }
    for k in kwargs:
        if k in ['max_tokens', 'temperature', 'top_p']:
            payload[k] = kwargs[k]
    response = predictor.predict(payload)
    
    if print_response:
        # Print usage statistics
        usage = response['usage']
        print(response['choices'][0]['message']['content'].strip())
        print(f"=== Token Usage: {usage['prompt_tokens']} (prompt), {usage['completion_tokens']} (completions), {usage['total_tokens']} (total) ===")
    return response['choices'][0]['message']['content']

In [26]:
_ = invoke_sealion_multimodal(
    prompt = "What animal is on the candy?",
    image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"
)

Based on the image, the animal on the candy appears to be a **turtle**. 

You can see the shape of a turtle shell and head on each of the candies.
=== Token Usage: 282 (prompt), 37 (completions), 319 (total) ===


### Streaming responses
You can also direclty stream response from your endpoint. To achieve this, we will use the invoke_endpoint_with_response_stream API.

You can **interleave images with text**. To do so, just cut off the input text where you want to insert an image, and insert it with an image block like the following.

In [27]:
from IPython.display import Image as IPyImage
# IPyImage(url="https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/IMG_3018.JPG", height=300, width= 300)
IPyImage(url="https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/fruit_knife.png", height=300, width= 300)

In [28]:
from IPython.display import Image as IPyImage
# IPyImage(url="https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/IMG_3015.jpg", height=300, width= 300)
IPyImage(url="https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/candy.JPG", height=300, width= 300)

In [29]:
body = {
  "messages": [
    {
      "role": "system",
      "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "I have this "},  # "I'm already using this supplement "},
        {
          "type": "image_url", 
          "image_url": {"url": "https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/candy.JPG"},
        },
        {"type": "text", "text": "and I want to use this as well "},
        {
          "type": "image_url", 
          "image_url": {"url": "https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/fruit_knife.png"}, # "https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/IMG_3015.jpg"},
        },
        {"type": "text", "text": " what are cautions?"},
      ]
    }
  ],
  "max_tokens": 1500,
  "temperature": 0.6,
  "top_p": 0.9,
  "stream": True
}

In [30]:
import json
import time

# Create SageMaker Runtime client
sagemaker_runtime_client = boto3.client("sagemaker-runtime")
print(f'Using endpoint: {endpoint_name}')

# Invoke the model
response_stream = sagemaker_runtime_client.invoke_endpoint_with_response_stream(
    EndpointName = endpoint_name,
    ContentType = "application/json",
    Body = json.dumps(body)
)

first_token_received = False
ttft = None
token_count = 0
start_time = time.time()

print("Response:", end=' ', flush=True)
full_response = ""

for event in response_stream['Body']:
    if 'PayloadPart' in event:
        chunk = event['PayloadPart']['Bytes'].decode()
        
        try:
            # Handle SSE format (data: prefix)
            if chunk.startswith('data: '):
                data = json.loads(chunk[6:])  # Skip "data: " prefix
            else:
                data = json.loads(chunk)
            
            # Extract token based on OpenAI format
            if 'choices' in data and len(data['choices']) > 0:
                if 'delta' in data['choices'][0] and 'content' in data['choices'][0]['delta']:
                    token_count += 1
                    token_text = data['choices'][0]['delta']['content']
                                    # Record time to first token
                    if not first_token_received:
                        ttft = time.time() - start_time
                        first_token_received = True
                    full_response += token_text
                    print(token_text, end='', flush=True)
        
        except json.JSONDecodeError:
            continue
            
# Print metrics after completion
end_time = time.time()
total_latency = end_time - start_time

print("\n\nMetrics:")
print(f"Time to First Token (TTFT): {ttft:.2f} seconds" if ttft else "TTFT: N/A")
print(f"Total Tokens Generated: {token_count}")
print(f"Total Latency: {total_latency:.2f} seconds")
if token_count > 0 and total_latency > 0:
    print(f"Tokens per second: {token_count/total_latency:.2f}")

Using endpoint: gemma-sea-lion-v4-27b-it-250901-0659
Response: Okay I see the images. You have a hand holding colorful, small candies (likely Jelly Belly brand, judging by the shape and design) and a picture of various fruits, a knife, and a cutting board. 

Here are some cautions, considering you might be thinking of combining these images or concepts:

**Regarding the Candies & Fruit:**

*   **Sugar Intake:** Combining images of candy with fruit could imply a focus on sugary foods. Be mindful of promoting a balanced diet. Too much sugar isn't healthy.
*   **Choking Hazard:** The candies are small. If you're using this in a context where young children might see it, it's important to note they could be a choking hazard.
*   **Misleading Association:** Don't present the candies *as if* they are a healthy alternative to fruit. They are very different nutritionally.

**Regarding the Knife & Fruit (and potentially the candies):**

*   **Sharp Objects:** The knife is a potential hazard. If

# Clean up

In [31]:
# Clean up
predictor.delete_model()
predictor.delete_endpoint(delete_endpoint_config=True)

Note that failure to delete the endpoint will incur costs.

The ml.g5.12xlarge and ml.g6e.48xlarge instances costs (for running 24/7 SageMaker Realtime Inference in us-east-1) are:

Instance Type     | Cost/hr   | Cost/day  | Cost/month
------------------|-----------|-----------|-------------
`ml.g5.12xlarge`  | USD  7.09 |USD 170.16 |USD  5,175.70
`ml.g6e.48xlarge` | USD 37.664|USD 903.936|USD 27,494.72

It is highly recommended that you delete endpoints that are no longer in use, or that you choose an option such as [SageMaker Scale down to zero feature](https://docs.aws.amazon.com/sagemaker/latest/dg/endpoint-auto-scaling-zero-instances.html) as mentioned in this blog post: [Unlock cost savings with the new scale down to zero feature in SageMaker Inference](https://aws.amazon.com/blogs/machine-learning/unlock-cost-savings-with-the-new-scale-down-to-zero-feature-in-amazon-sagemaker-inference/).

In [ ]:
# Use this to generate html version of the Jupyter notebook
!jupyter nbconvert Gemma-SEA-LION-v4-27B-IT.ipynb --to html